In [1]:
import pandas as pd

In [3]:
df = pd.read_excel(
    "/Users/abhinavkashyap/abhi/projects/headlamphealth/headlamphealth/sample_data/patients_data.xlsx"
)

In [6]:
df.head()

,Unnamed: 0,date,sleep,diet,social,exercise,anxious,happy,motivation
0,20,2023-10-30,20,20.0,10.0,20,9,0,0
1,21,2023-10-31,20,20.0,20.0,20,6,0,0
2,22,2023-11-02,20,30.0,20.0,20,2,3,3
3,23,2023-11-05,20,30.0,30.0,10,8,2,3
4,24,2023-11-07,30,20.0,30.0,10,8,2,3


In [8]:
# Let us get some information about the dataset
num_rows = len(df)
print(f"Number of rows in the {num_rows}")

# Find the number of missing values in each column in the dataset
unique_values = {col: df[col].unique().tolist() for col in df.columns}

for col, values in unique_values.items():
    print(f"Unique Values in col {col}: {values}")

Number of rows in the 91
Unique Values in col Unnamed: 0: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280]
Unique Values in col date: [Timestamp('2023-10-30 00:00:00'), Timestamp('2023-10-31 00:00:00'), Timestamp('2023-11-02 00:00:00'), Timestamp('2023-11-05 00:00:00'), Timestamp('2023-11-07 00:00:00'), Timestamp('2023-11-08 00:00:00'), Timestamp('2023-11-10 00:00:00'), Timestamp('2023-11-12 00:00:00'), Timestamp('2023-11-13 00:00:00'), Timestamp('2023-11-16 00:00:00'), Timestamp('2023-11-19 00:00:00'), Timestamp('2023-11-23 00:00:00'), Timestamp('2023-11-24 00:00:00'), Timestamp('2023-11-27 00:00:00'), Timestamp('2023-12-01 00:00:00'), T

In [11]:
# HOw many values in each column are nan
# There is a chance that a person may log only some of the values in the column
num_nan = {col: df[col].isna().sum() for col in df.columns}
for col, count in num_nan.items():
    print(f"Number of nans in {col}: {count}")


# Social seems to be something that is not logged by the patient at all
# For some of the things like diet, we can assume that it does not change too much and fill the NAs with mean value
# But for others like social, it can vary quite a bit from person to person.

Number of nans in Unnamed: 0: 0
Number of nans in date: 0
Number of nans in sleep: 0
Number of nans in diet: 38
Number of nans in social: 38
Number of nans in exercise: 0
Number of nans in anxious: 0
Number of nans in happy: 0
Number of nans in motivation: 0


In [17]:
# let us find the correlation between anxiety and sleep
# I would expect, lesser the sleep more the anxiety. Is this the case?

sleep_anxiety_corr = df["anxious"].corr(df["sleep"])
sleep_exercise_corr = df["anxious"].corr(df["exercise"])

# Greater the sleep, lesser the anxiety
print(f"Sleep-Anxiety Correlation {sleep_anxiety_corr}")

# Greater the exercise, lesser the anxiety as well
print(f"Exercise-Anxiety Correlation {sleep_exercise_corr}")

# We can do similar for other mental-health measures as well

sleep_happy_corr = df["happy"].corr(df["sleep"])
print(f"Sleep-Happy Correlation {sleep_happy_corr}")
exercise_happy_corr = df["happy"].corr(df["exercise"])
print(f"Exercise-Happy Correlation {exercise_happy_corr}")

Sleep-Anxiety Correlation -0.27452726835962865
Exercise-Anxiety Correlation -0.5833844944308754
Sleep-Happy Correlation 0.15899145036459747
Exercise-Happy Correlation 0.3331401959190728


In [19]:
# These correlations seem to agree with common knowledge about mental health
# We can build the linear regression that


# This can horribly be wrong. This should be checked.
# The other way to do it is to drop the rows that have incomplete values
# for demonstration purposes, let's do this in the first place.
for col in ["diet", "social"]:
    df[col] = df[col].fillna(df[col].mean())

In [21]:
# Check whether the diet and social do not have nas
print(df["diet"].isna().sum())
print(df["social"].isna().sum())

0
0


In [22]:
# lets perform a simple linear regression with anxiety as the dependent variable
# The other columns -- sleep, diet, social, exercise are going to be the independent variable

from sklearn.linear_model import LinearRegression

X = df[["sleep", "diet", "social", "exercise"]]
y = df["anxious"]

# WE can split this into 80 10 10 train validation and test split first before fitting
# a linear model
reg = LinearRegression().fit(X, y)

In [24]:
# This is the coefficient of regression which is the R2
# This checks whether the fitted values capture the variation in the dependent variable
# WE get around 0.4 which is not too high actually
# But given that there is only 92 instances, we cannot expect a really good linear classifier
reg.score(X, y)

0.4144942269001145